## Normal Abnormal binary classification model

In [1]:
cd /home/ines/code/ISOCC/data/

/home/ines/code/ISOCC/data


In [49]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.signal import stft

import librosa
import librosa.display

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torch.nn.functional as F

## Hyper-param

In [54]:
batch_size = 32
epochs = 100

In [3]:
abnormal_df = pd.read_csv("ptbdb_abnormal.csv", header=None)
normal_df = pd.read_csv("ptbdb_normal.csv", header=None)

print(normal_df.shape)
abnormal_df.shape

(4046, 188)


(10506, 188)

In [4]:
ECG_df = pd.concat([normal_df, abnormal_df])
#ECG_df.columns = ECG_df.columns.astype("str")
ECG_df[187] = ECG_df[187].astype("int64")
print(ECG_df.head())
print("shape: ", ECG_df.shape)
ECG_df.info()

        0         1         2         3         4         5         6    \
0  1.000000  0.900324  0.358590  0.051459  0.046596  0.126823  0.133306   
1  1.000000  0.794681  0.375387  0.116883  0.000000  0.171923  0.283859   
2  0.909029  0.791482  0.423169  0.186712  0.000000  0.007836  0.063032   
3  1.000000  0.478893  0.056760  0.064176  0.081289  0.072732  0.055619   
4  1.000000  0.867238  0.201360  0.099349  0.141336  0.120934  0.108516   

        7         8         9    ...  178  179  180  181  182  183  184  185  \
0  0.119125  0.110616  0.113047  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
1  0.293754  0.325912  0.345083  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
2  0.077002  0.074957  0.077342  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
3  0.048774  0.054478  0.041643  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
4  0.096393  0.093436  0.100828  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

   186  187  
0  0.0    0  
1  0.0    0  
2  0.0    0  
3  0.0    0 

In [5]:
label = {1: "Normal", 0: "Abnormal"}
X = ECG_df.loc[:, :186].to_numpy()
y = ECG_df.loc[:, 187:187].to_numpy()

print(X.shape, y.shape)

(14552, 187) (14552, 1)


In [7]:
for i in range(len(X)):
    plt.plot(range(len(X[i])), X[i])
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('ECG Data')

    # 이미지 저장
    save_dir = "/home/ines/code/ISOCC/wav_data"
    plt.savefig(os.path.join(save_dir, 'ecg_plot{}.png'.format(i)))
    plt.close()
    if i % 1000 == 0:
        print(i)

In [13]:
# STFT 저장

fs = 250  # 샘플링 빈도
nperseg = 4  # 세그먼트당 샘플 수
noverlap = nperseg // 2  # 중첩 샘플 수
nfft = 8  # FFT 포인트 수

save_path = '/home/ines/code/ISOCC/stft_data/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 각 시리즈에 대해 STFT 실행 및 결과 저장
i = 0
for column in X:
    f, t, Zxx = stft(column, fs=fs, nperseg=nperseg, noverlap=noverlap, nfft=nfft)
    np.save(os.path.join(save_path, f'ECG_STFT_{i}.npy'), Zxx)
    if i % 1000 == 0:
        print(i)
    i += 1

# 저장된 파일의 예시 경로 확인
f'Files saved in {save_path}'

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


'Files saved in /home/ines/code/ISOCC/stft_data/'

In [73]:
file_paths = [os.path.join(save_path, f) for f in os.listdir(save_path) if f.endswith('.npy')]
data = [np.load(f) for f in file_paths]

tensor_data = [torch.tensor(d, dtype=torch.float32) for d in data]
print(tensor_data[1].shape)

torch.Size([5, 95])


In [64]:
X_train, X_test, y_train, y_test = train_test_split(tensor_data, y, train_size=0.8, random_state=42
print(y_train.shape, y_test.shape)

(11641, 1) (2911, 1)


In [67]:
class CustomDataset(Dataset):
    def __init__(self, datas, label_arr):
        self.datas = datas
        self.label_arr = label_arr

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, idx):
        data = self.datas[idx]
        label = self.label_arr[idx]
        return data, label

train_dataset = CustomDataset(X_train, y_train)
val_dataset = CustomDataset(X_test, y_test)

train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=batch_size)

In [43]:
# 다른 논문에서 .npy 불러올 때 사용한 코드
# 참고용
'''
class TrainSet(Dataset):
    def __init__(self, folder):
        self.train_data = np.load(os.path.join(folder, 'train.npy'))

    # 주어진 ECG 신호에 대해 R 피크를 검출하는 함수, HeartPy 라이브러리를 사용하여 ECG 신호를 처리하고, 작업 데이터와 측정값을 반환
    def checkR(self, ecg):
        working_data, measures = hp.process(ecg, 500.0)
        peak_list = working_data['peaklist']
        return peak_list
    
    # 학습 데이터의 총 개수를 반환
    def __len__(self):
        return self.train_data.shape[0]
 
    def __getitem__(self, index):
    # 주어진 인덱스에 해당하는 학습 데이터(time_instance)를 가져옵니다.
    # 가져온 학습 데이터에서 첫 100개와 마지막 100개의 데이터를 제거하여 4900개의 데이터 포인트를 남깁니다. 이로써 총 4800개의 데이터 포인트로 이루어진 시간축 데이터
        time_instance = self.train_data[index]
        time_instance = time_instance[100:4900,:] #(4800, 12)
       
        # Short Time Fast Fourier Transform
        # 500 is the sample rate of PTB-XL, 360 is the sample rate of MIT-BIH
        # 스펙트로그램 데이터의 축을 재배치하여 (63, 78, 12)의 형태로 만듭니다. 여기서 (63, 78)는 STFT 결과의 스펙트로그램 크기이고, 12는 채널의 개수
        f,t, Zxx = stft(time_instance.transpose(1,0),fs=500, window='hann',nperseg=125)
        spectrogram_instance = np.abs(Zxx)  #(12, 63, 78)
        spectrogram_instance = spectrogram_instance.transpose(1,2,0)     #(63, 78, 12)
        return time_instance, spectrogram_instance
 
 
 
class TestSet(Dataset):
    def __init__(self, folder):
        self.test_data = np.load(os.path.join(folder, 'test.npy'))
       
    def __len__(self):
        return self.test_data.shape[0]
    
    def checkR(self, ecg):
        working_data, measures = hp.process(ecg, 500.0)
        peak_list = working_data['peaklist']
        return np.array(peak_list)
   
    def __getitem__(self, index):
        time_instance = self.test_data[index]
        r_index = self.checkR(time_instance[:,1])
        time_instance = time_instance[100:4900,:]
        # Short Time Fast Fourier Transform
        f,t, Zxx = stft(time_instance.transpose(1,0),fs=500, window='hann',nperseg=125)
        spectrogram_instance = np.abs(Zxx)  #(12, 63, 78)
        spectrogram_instance = spectrogram_instance.transpose(1,2,0)     #(63, 78, 12)
        return time_instance, spectrogram_instance, r_index
 '''

- 입력값 = (batch_size, 5, 95)

## BCResNet

In [71]:
# Copyright (c) 2023 Qualcomm Technologies, Inc.
# All Rights Reserved.

class SubSpectralNorm(nn.Module):
    def __init__(self, num_features, spec_groups=16, affine="Sub", batch=True, dim=2):
        super().__init__()
        self.spec_groups = spec_groups
        self.affine_all = False
        affine_norm = False
        if (
            affine == "Sub"
        ):  # affine transform for each sub group. use affine of torch implementation
            affine_norm = True
        elif affine == "All":
            self.affine_all = True
            self.weight = nn.Parameter(torch.ones((1, num_features, 1, 1)))
            self.bias = nn.Parameter(torch.zeros((1, num_features, 1, 1)))
        if batch:
            self.ssnorm = nn.BatchNorm2d(num_features * spec_groups, affine=affine_norm)
        else:
            self.ssnorm = nn.InstanceNorm2d(num_features * spec_groups, affine=affine_norm)
        self.sub_dim = dim

    def forward(self, x):  # when dim h is frequency dimension
        if self.sub_dim in (3, -1):
            x = x.transpose(2, 3)
            x = x.contiguous()
        b, c, h, w = x.size()
        assert h % self.spec_groups == 0
        x = x.view(b, c * self.spec_groups, h // self.spec_groups, w)
        x = self.ssnorm(x)
        x = x.view(b, c, h, w)
        if self.affine_all:
            x = x * self.weight + self.bias
        if self.sub_dim in (3, -1):
            x = x.transpose(2, 3)
            x = x.contiguous()
        return x

In [41]:
# Copyright (c) 2023 Qualcomm Technologies, Inc.
# All Rights Reserved.

import torch.nn.functional as F
from torch import nn

class ConvBNReLU(nn.Module):
    def __init__(
        self,
        in_plane,
        out_plane,
        idx,
        kernel_size=3,
        stride=1,
        groups=1,
        use_dilation=False,
        activation=True,
        swish=False,
        BN=True,
        ssn=False,
    ):
        super().__init__()

        def get_padding(kernel_size, use_dilation):
            rate = 1  # dilation rate
            padding_len = (kernel_size - 1) // 2
            if use_dilation and kernel_size > 1:
                rate = int(2**self.idx)
                padding_len = rate * padding_len
            return padding_len, rate

        self.idx = idx

        # padding and dilation rate
        if isinstance(kernel_size, (list, tuple)):
            padding = []
            rate = []
            for k_size in kernel_size:
                temp_padding, temp_rate = get_padding(k_size, use_dilation)
                rate.append(temp_rate)
                padding.append(temp_padding)
        else:
            padding, rate = get_padding(kernel_size, use_dilation)

        # convbnrelu block
        layers = []
        layers.append(
            nn.Conv2d(in_plane, out_plane, kernel_size, stride, padding, rate, groups, bias=False)
        )
        if ssn:
            layers.append(SubSpectralNorm(out_plane, 5))
        elif BN:
            layers.append(nn.BatchNorm2d(out_plane))
        if swish:
            layers.append(nn.SiLU(True))
        elif activation:
            layers.append(nn.ReLU(True))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


class BCResBlock(nn.Module):
    def __init__(self, in_plane, out_plane, idx, stride):
        super().__init__()
        self.transition_block = in_plane != out_plane
        kernel_size = (3, 3)

        # 2D part (f2)
        layers = []
        if self.transition_block:
            layers.append(ConvBNReLU(in_plane, out_plane, idx, 1, 1))
            in_plane = out_plane
        layers.append(
            ConvBNReLU(
                in_plane,
                out_plane,
                idx,
                (kernel_size[0], 1),
                (stride[0], 1),
                groups=in_plane,
                ssn=True,
                activation=False,
            )
        )
        self.f2 = nn.Sequential(*layers)
        self.avg_gpool = nn.AdaptiveAvgPool2d((1, None))

        # 1D part (f1)
        self.f1 = nn.Sequential(
            ConvBNReLU(
                out_plane,
                out_plane,
                idx,
                (1, kernel_size[1]),
                (1, stride[1]),
                groups=out_plane,
                swish=True,
                use_dilation=True,
            ),
            nn.Conv2d(out_plane, out_plane, 1, bias=False),
            nn.Dropout2d(0.1),
        )

    def forward(self, x):
        # 2D part
        shortcut = x
        x = self.f2(x)
        aux_2d_res = x
        x = self.avg_gpool(x)

        # 1D part
        x = self.f1(x)
        x = x + aux_2d_res
        if not self.transition_block:
            x = x + shortcut
        x = F.relu(x, True)
        return x


def BCBlockStage(num_layers, last_channel, cur_channel, idx, use_stride):
    stage = nn.ModuleList()
    channels = [last_channel] + [cur_channel] * num_layers
    for i in range(num_layers):
        stride = (2, 1) if use_stride and i == 0 else (1, 1)
        stage.append(BCResBlock(channels[i], channels[i + 1], idx, stride))
    return stage


class BCResNets(nn.Module):
    def __init__(self, base_c, num_classes=12):
        super().__init__()
        self.num_classes = num_classes
        self.n = [2, 2, 4, 4]  # identical modules repeated n times
        self.c = [
            base_c * 2,
            base_c,
            int(base_c * 1.5),
            base_c * 2,
            int(base_c * 2.5),
            base_c * 4,
        ]  # num channels
        self.s = [1, 2]  # stage using stride
        self._build_network()

    def _build_network(self):
        # Head: (Conv-BN-ReLU)
        self.cnn_head = nn.Sequential(
            nn.Conv2d(1, self.c[0], 5, (2, 1), 2, bias=False),
            nn.BatchNorm2d(self.c[0]),
            nn.ReLU(True),
        )
        # Body: BC-ResBlocks
        self.BCBlocks = nn.ModuleList([])
        for idx, n in enumerate(self.n):
            use_stride = idx in self.s
            self.BCBlocks.append(BCBlockStage(n, self.c[idx], self.c[idx + 1], idx, use_stride))

        # Classifier
        self.classifier = nn.Sequential(
            nn.Conv2d(
                self.c[-2], self.c[-2], (5, 5), bias=False, groups=self.c[-2], padding=(0, 2)
            ),
            nn.Conv2d(self.c[-2], self.c[-1], 1, bias=False),
            nn.BatchNorm2d(self.c[-1]),
            nn.ReLU(True),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(self.c[-1], self.num_classes, 1),
        )

    def forward(self, x):
        x = self.cnn_head(x)
        for i, num_modules in enumerate(self.n):
            for j in range(num_modules):
                x = self.BCBlocks[i][j](x)
        x = self.classifier(x)
        x = x.view(-1, x.shape[1])
        return x